In [8]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive/drive/')

Mounted at /content/drive


In [10]:
cd drive/MyDrive/Handwriting_recog_train

/content/drive/MyDrive/Handwriting_recog_train


In [11]:
ls

annotations.csv  dataset/


In [12]:
import cv2
import os
import csv
import numpy as np
import shutil
from skimage.feature import hog
from skimage.morphology import skeletonize
from skimage import exposure

import numpy as np
import tensorflow as tf
from keras import layers, models, Model, Input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [39]:
# Folder paths
annotation_csv = 'annotations.csv'
dataset_folder = 'dataset'  # Folder containing original images
# dataset_folder = 'drive/MyDrive/Handwriting_recog_train'  # Folder containing original images

# Alpha
# alpha = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# Load annotations
with open(annotation_csv, 'r') as file:
    reader = csv.reader(file)
    annotations = list(reader)[1:]  # Skip header


In [31]:
annotations

[['dataset\\a1.jpg', 'a', '299', '375', '50', '70'],
 ['dataset\\a1.jpg', 'a', '440', '383', '50', '70'],
 ['dataset\\a1.jpg', 'a', '588', '370', '77', '108'],
 ['dataset\\a1.jpg', 'a', '775', '378', '77', '108'],
 ['dataset\\a1.jpg', 'a', '910', '384', '77', '108'],
 ['dataset\\a1.jpg', 'a', '1085', '389', '66', '94'],
 ['dataset\\a1.jpg', 'a', '1306', '381', '66', '94'],
 ['dataset\\a1.jpg', 'a', '1443', '383', '66', '93'],
 ['dataset\\a1.jpg', 'a', '1630', '377', '66', '94'],
 ['dataset\\a1.jpg', 'a', '1758', '363', '66', '93'],
 ['dataset\\a1.jpg', 'a', '1873', '361', '66', '94'],
 ['dataset\\a1.jpg', 'A', '282', '459', '67', '93'],
 ['dataset\\a1.jpg', 'A', '426', '458', '67', '93'],
 ['dataset\\a1.jpg', 'A', '575', '470', '66', '93'],
 ['dataset\\a1.jpg', 'A', '759', '471', '66', '93'],
 ['dataset\\a1.jpg', 'A', '927', '478', '66', '93'],
 ['dataset\\a1.jpg', 'A', '1087', '478', '66', '93'],
 ['dataset\\a1.jpg', 'A', '1291', '471', '63', '88'],
 ['dataset\\a1.jpg', 'A', '1409', '

In [37]:
ls

annotations.csv  dataset/


In [43]:
blah = cv2.imread("dataset/a1.jpg")
if blah is None:
    print("blah")

In [44]:
flag = annotations[0][0]
c = 0
image_list1, image_list2, image_list3, image_list4, image_list5 = [], [], [], [], []
label_list = []

# Feature extraction steps
for annotation in annotations:
    c += 1
    image_name1, letter, center_x, center_y, dist_x, dist_y = annotation

    # Load original image
    image_name1 = image_name1.replace('\\', '/')
    # image_path = os.path.join(dataset_folder, image_name)
    # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.imread(image_name1, cv2.IMREAD_GRAYSCALE)
    image_name = image_name1[image_name1.find('\\')+1:image_name1.find('.')]

    if img is None:
        # print(f"Error: {image_path}_{c} not found!")
        print(f"Error: {image_name1}_{c} not found!")
        continue

    # Crop the letter using the annotation box
    start_x = int(float(center_x) - float(dist_x) / 2)
    start_y = int(float(center_y) - float(dist_y) / 2)
    end_x = int(float(center_x) + float(dist_x) / 2)
    end_y = int(float(center_y) + float(dist_y) / 2)
    cropped = img[start_y:end_y, start_x:end_x]

    # 1. Image Enhancement & Normalization (Histogram Equalization)
    resized = cv2.resize(cropped, (64, 64))
    enhanced = cv2.equalizeHist(resized)
    norm_image = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
    image_list1.append(norm_image)
    label_list.append(letter)

    # 2. Segmentation (Thresholding)
    _, segmented = cv2.threshold(norm_image, 30, 255, cv2.THRESH_BINARY_INV)
    image_list2.append(segmented)

    # 3. Edge Detection (Canny)
    edges = cv2.Canny(segmented, 100, 200)
    image_list3.append(edges)

    # 4. Skeletonization
    binary = segmented / 255  # Convert to binary (0, 1)
    skeleton = skeletonize(binary).astype(np.uint8) * 255
    image_list4.append(skeleton)

    # 5. HOG (Histogram of Oriented Gradients)
    hog_features, hog_image = hog(segmented, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=True)
    image_list5.append(hog_image)


In [45]:
label_list

['a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'b',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'd',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'e',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'f',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'g',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'h',
 'H',
 'H'

In [46]:
alphaIndexes = {}
for i in alpha:
    alphaIndexes[i] = []
for i in range(len(label_list)):
    try:
        alphaIndexes[label_list[i]].append(i)
    except:
        pass
# alphaIndexes

In [47]:
indexes = [i for i in range(len(label_list))]
xrl, xtl = train_test_split(indexes, test_size=0.2, random_state=42)

In [48]:
splitIndexes = {}
for i in alpha:
    xrl, xtl = train_test_split(alphaIndexes[i], test_size=0.2, random_state=42)
    # print(i, len(xrl), len(xtl), sep=", ")
    times = 500//len(xrl)
    print(times, end=" ")
    xrl1 = xrl.copy()
    for j in range(times): xrl1.extend(xrl)
    print(i, len(xrl1), len(xtl), sep=", ")
    splitIndexes[i] = [sorted(xrl1), sorted(xtl)]
# splitIndexes

2 a, 735, 62
6 b, 532, 20
3 c, 568, 36
2 d, 525, 44
1 e, 810, 102
11 f, 516, 11
5 g, 528, 22
3 h, 536, 34
2 i, 720, 61
50 j, 510, 3
15 k, 512, 8
4 l, 580, 29
4 m, 535, 27
1 n, 636, 80
1 o, 602, 76
5 p, 600, 25
22 q, 506, 6
2 r, 621, 52
2 s, 702, 59
1 t, 580, 73
4 u, 615, 31
10 v, 528, 12
9 w, 520, 13
22 x, 506, 6
7 y, 504, 16
33 z, 510, 4


In [49]:
xrl, xtl = [], []
for i in alpha:
    xrl.extend(splitIndexes[i][0])
    xtl.extend(splitIndexes[i][1])

In [50]:
xrl

[0,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 4,
 4,
 4,
 6,
 6,
 6,
 8,
 8,
 8,
 10,
 10,
 10,
 595,
 595,
 595,
 611,
 611,
 611,
 631,
 631,
 631,
 643,
 643,
 643,
 646,
 646,
 646,
 661,
 661,
 661,
 687,
 687,
 687,
 692,
 692,
 692,
 706,
 706,
 706,
 717,
 717,
 717,
 721,
 721,
 721,
 727,
 727,
 727,
 808,
 808,
 808,
 814,
 814,
 814,
 822,
 822,
 822,
 826,
 826,
 826,
 840,
 840,
 840,
 845,
 845,
 845,
 856,
 856,
 856,
 912,
 912,
 912,
 919,
 919,
 919,
 929,
 929,
 929,
 944,
 944,
 944,
 957,
 957,
 957,
 974,
 974,
 974,
 985,
 985,
 985,
 989,
 989,
 989,
 1006,
 1006,
 1006,
 1010,
 1010,
 1010,
 1044,
 1044,
 1044,
 1048,
 1048,
 1048,
 1051,
 1051,
 1051,
 1056,
 1056,
 1056,
 1062,
 1062,
 1062,
 1145,
 1145,
 1145,
 1163,
 1163,
 1163,
 1178,
 1178,
 1178,
 1190,
 1190,
 1190,
 1193,
 1193,
 1193,
 1198,
 1198,
 1198,
 1212,
 1212,
 1212,
 1229,
 1229,
 1229,
 1238,
 1238,
 1238,
 1307,
 1307,
 1307,
 1312,
 1312,
 1312,
 1332,
 1332,
 1332,
 1357,
 1357,
 1357,
 1418,


In [51]:
def lettersSplit(imgList, labelList, uniqueLabels=alpha):
    labelToIdx = {label: idx for idx, label in enumerate(uniqueLabels)}
    numericalLabel = np.array([labelToIdx[label] for label in labelList])

    # Split data into training and testing sets
    x_train, x_test, y_train, y_test = [imgList[i] for i in xrl], [imgList[i] for i in xtl], [numericalLabel[i] for i in xrl], [numericalLabel[i] for i in xtl]

    # Normalize the image data (scale pixel values to range [0, 1])
    x_train = np.array(x_train) / 255.0
    x_test = np.array(x_test) / 255.0

    # Convert labels to categorical (one-hot encoding)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(uniqueLabels))
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(uniqueLabels))
    x_train = np.expand_dims(x_train, axis=-1)  # (num_samples, height, width, 1)
    x_test = np.expand_dims(x_test, axis=-1)

    # Set input shape based on your image data (e.g., (height, width, channels))
    input_shape = x_train.shape[1:]  # Height, Width, Channels
    return input_shape, x_train, x_test, y_train, y_test

In [60]:
# Define the CNN model
def cnnTraining(input_shape):
    # First convolutional layer
    x = layers.Conv2D(64, (3,3), activation = 'relu')(input_shape)
    x = layers.MaxPooling2D((2,2))(x)

    # Second convolutional layer
    x = layers.Conv2D(128,(3, 3), activation = 'relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Third convolutional layer
    x = layers.Conv2D(64, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Flatten the output
    x = layers.Flatten()(x)
    return x

In [53]:
unique_labels = sorted(list(set(label_list)))  # Get unique classes
num_classes = len(unique_labels)

input_shape1, x_train_1, x_test_1, y_train_1, y_test_1 = lettersSplit(image_list1, label_list, unique_labels)
input_shape2, x_train_2, x_test_2, y_train_2, y_test_2 = lettersSplit(image_list2, label_list, unique_labels)
input_shape3, x_train_3, x_test_3, y_train_3, y_test_3 = lettersSplit(image_list3, label_list, unique_labels)
input_shape4, x_train_4, x_test_4, y_train_4, y_test_4 = lettersSplit(image_list4, label_list, unique_labels)
input_shape5, x_train_5, x_test_5, y_train_5, y_test_5 = lettersSplit(image_list5, label_list, unique_labels)


In [61]:
input1 = Input(shape = input_shape1)
input2 = Input(shape= input_shape2)
input3 = Input(shape= input_shape3)
input4 = Input(shape= input_shape4)
input5 = Input(shape= input_shape5)

# Create the model
cnn1 = cnnTraining(input1)
cnn2 = cnnTraining(input2)
cnn3 = cnnTraining(input3)
cnn4 = cnnTraining(input4)
cnn5 = cnnTraining(input5)

concatenated = layers.Concatenate()([cnn1, cnn2, cnn3, cnn4, cnn5])

x = layers.Dense(128, activation = 'relu')(concatenated)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=[input1, input2, input3, input4, input5], outputs = output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 64, 64, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_11            │ (None, 64, 64, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_12            │ (None, 64, 64, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_13            │ (None, 64, 64, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_14            │ (None, 64, 64, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_30 (Conv2D)        │ (None, 62, 62, 64)     │            640 │ input_layer_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_33 (Conv2D)        │ (None, 62, 62, 64)     │            640 │ input_layer_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_36 (Conv2D)        │ (None, 62, 62, 64)     │            640 │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_39 (Conv2D)        │ (None, 62, 62, 64)     │            640 │ input_layer_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_42 (Conv2D)        │ (None, 62, 62, 64)     │            640 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_30          │ (None, 31, 31, 64)     │              0 │ conv2d_30[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_33          │ (None, 31, 31, 64)     │              0 │ conv2d_33[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_36          │ (None, 31, 31, 64)     │              0 │ conv2d_36[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_39          │ (None, 31, 31, 64)     │              0 │ conv2d_39[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_42          │ (None, 31, 31, 64)     │              0 │ conv2d_42[0][0]        │
│ (MaxPooling2D)       

 Total params: 2,227,764 (8.50 MB)

 Trainable params: 2,227,764 (8.50 MB)

 Non-trainable params: 0 (0.00 B)

In [55]:
x_train_1.shape

(15037, 64, 64, 1)

In [62]:
history = model.fit([x_train_1, x_train_2, x_train_3, x_train_4, x_train_5], [y_train_1, y_train_2, y_train_3, y_train_4, y_train_5], epochs = 20, batch_size=32, validation_data=([x_test_1, x_test_2, x_test_3, x_test_4, x_test_5], [y_test_1, y_test_2, y_test_3, y_test_4, y_test_5]))

test_loss, test_acc = model.evaluate([x_test_1, x_test_2, x_test_3, x_test_4, x_test_5], [y_test_1, y_test_2, y_test_3, y_test_4, y_test_5])
print(f"Test accuracy: {test_acc}")

Epoch 1/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 29s 45ms/step - accuracy: 0.4561 - loss: 1.9721 - val_accuracy: 0.6338 - val_loss: 1.1863
Epoch 2/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 32s 33ms/step - accuracy: 0.9311 - loss: 0.2277 - val_accuracy: 0.7204 - val_loss: 1.2513
Epoch 3/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9779 - loss: 0.0710 - val_accuracy: 0.6787 - val_loss: 1.7408
Epoch 4/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.9817 - loss: 0.0570 - val_accuracy: 0.7094 - val_loss: 1.8159
Epoch 5/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.9874 - loss: 0.0361 - val_accuracy: 0.6776 - val_loss: 1.9457
Epoch 6/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9895 - loss: 0.0358 - val_accuracy: 0.6886 - val_loss: 2.1072
Epoch 7/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.9906 - loss: 0.0329 - val_accuracy: 0.6996 - val_loss: 1.9893
Epoch 8/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 16s 33ms/step - accuracy: 0.9946 - loss: 0.0162 - 